In [ ]:
#Our project is two steps:
#1. Detect if a given package has suspicious activity (a peak or a step in downloads)
#2. Determine whether that suspicious activity can be explained.

#Step 1 is doing the "magic math" and stuff on our pkg_list. (Already completed, but needs further validation)

#Step 2 has many parts. 
#2.1 Was the package added or dropped as a dependency on a popular project around the time of the rise/ fall of downloads? (Next step)
#2.2 Did any dependents have a similar rise/ fall in downloads? (In progress, but is proving very unreliable so may be dropped). 
#2.2 is probably as far as we will be able to make it, but we can discuss further steps in our presentation
#2.3 Did any developer/ nightly builds have our package as a dependency? 
#2.4 Did a new version release that correlates to increased downloads?
#Etc

In [1]:
import gzip, json
with gzip.open('/data/NPMvulnerabilities/NPMPackages.json.gz', 'rb') as f:
    fc = f.read().decode("utf-8")
from json import JSONDecoder, JSONDecodeError
import re

NOT_WHITESPACE = re.compile(r'[^\s]')

def decode_stacked(document, pos=0, decoder=JSONDecoder()):
    while True:
        match = NOT_WHITESPACE.search(document, pos)
        if not match:
            return
        pos = match.start()

        try:
            obj, pos = decoder.raw_decode(document, pos)
        except JSONDecodeError:
            # do something sensible if there's some error
            print("DECODE_STACKED() FAILED")
            pass
        yield obj

In [ ]:
for k in dependent.keys():
    if k == 'ajv-keywords':
        print(k, dependent[k][0])
        break

In [165]:
#, 'user-info', 'exec-file-sync', 'babylon-walk', 'speedtest-net', 'dns-then', 'node-then', '@babel/polyfill', '@babel/runtime', 'dvp-mongomodels', '@babel/helper-builder-react-jsx', 'doofinder', 'highland-errors-to', 'highland-process', '@cnbritain/merlin-www-section-card-list', '@cnbritain/merlin-www-tag-header', '@cnbritain/merlin-www-card-list', '@cnbritain/merlin-www-top-stories', 'uapi-json', '@beisen/ethos', '@cnbritain/merlin-www-card', '@babel/helper-replace-supers', 'bfj-node4', 'highland-errors-to', 'highland-process', '@particles/spinner', 'cli-ngy', '@cnbritain/merlin-www-card', 'neutrino-middleware-named-modules', 'chrome-pool', '@cnbritain/merlin-www-topic-strip', 'awayjs-display', '@babel/helper-replace-supers', 'cloudcms-server', 'authenticator-cli', 'authenticator', 'bfj-node4', 'chrome-runner', 'pino-toke', 'stencila', 'sqlite3-transactions', 'domotz-ip-address', 'authenticator-cli', 'safe-regex', 'easynode', 'authenticator', 'p-map-series', 'chunky-stream', 'expresser', 'semver-intersect', 'webpack-cli'



white_pkgs = ['ajv-keywords', 'htmlnano', 'object.map', 'babylon-walk', 'wix-style-react', 'threads', 'cloudscraper', 'user-info', 'exec-file-sync', 'babylon-walk', 'speedtest-net', 'dns-then', 'node-then', 'doofinder', 'highland-errors-to', 'highland-process', 'highland-errors-to', 'highland-process', 'chrome-pool', 'awayjs-display', 'cloudcms-server', 'authenticator-cli', 'authenticator', 'chrome-runner', 'pino-toke', 'domotz-ip-address', 'authenticator-cli', 'safe-regex', 'authenticator', 'p-map-series', 'semver-intersect']


In [2]:
#pkgs = ['object.map', 'babylon-walk', 'wix-style-react', 'threads', 'minimist-options', 'lead', '@babel/polyfill', '@babel/helper-define-map', 'cloudscraper']#, 'user-info', 'exec-file-sync', 'babylon-walk', 'speedtest-net', 'dns-then', 'node-then', '@babel/polyfill', '@babel/runtime', 'dvp-mongomodels', '@babel/helper-builder-react-jsx', 'doofinder', 'highland-errors-to', 'highland-process', '@cnbritain/merlin-www-section-card-list', '@cnbritain/merlin-www-tag-header', '@cnbritain/merlin-www-card-list', '@cnbritain/merlin-www-top-stories', 'uapi-json', '@beisen/ethos', '@cnbritain/merlin-www-card', '@babel/helper-replace-supers', 'bfj-node4', 'highland-errors-to', 'highland-process', '@particles/spinner', 'cli-ngy', '@cnbritain/merlin-www-card', 'neutrino-middleware-named-modules', 'chrome-pool', '@cnbritain/merlin-www-topic-strip', 'awayjs-display', '@babel/helper-replace-supers', 'cloudcms-server', 'authenticator-cli', 'authenticator', 'bfj-node4', 'chrome-runner', 'pino-toke', 'stencila', 'sqlite3-transactions', 'domotz-ip-address', 'authenticator-cli', 'safe-regex', 'easynode', 'authenticator', 'p-map-series', 'chunky-stream', 'expresser', 'semver-intersect', 'webpack-cli']
pkgs = ['user-info', 'exec-file-sync', 'babylon-walk', 'speedtest-net', 'dns-then', 'node-then', '@babel/polyfill', '@babel/runtime', 'dvp-mongomodels', '@babel/helper-builder-react-jsx', 'doofinder', 'highland-errors-to', 'highland-process', '@cnbritain/merlin-www-section-card-list', '@cnbritain/merlin-www-tag-header', '@cnbritain/merlin-www-card-list', '@cnbritain/merlin-www-top-stories', 'uapi-json', '@beisen/ethos', '@cnbritain/merlin-www-card', '@babel/helper-replace-supers', 'bfj-node4', 'highland-errors-to', 'highland-process', '@particles/spinner', 'cli-ngy', '@cnbritain/merlin-www-card', 'neutrino-middleware-named-modules', 'chrome-pool', '@cnbritain/merlin-www-topic-strip', 'awayjs-display', '@babel/helper-replace-supers', 'cloudcms-server', 'authenticator-cli', 'authenticator', 'bfj-node4', 'chrome-runner', 'pino-toke', 'stencila', 'sqlite3-transactions', 'domotz-ip-address', 'authenticator-cli', 'safe-regex', 'easynode', 'authenticator', 'p-map-series', 'chunky-stream', 'expresser', 'semver-intersect', 'webpack-cli']
#pkgs = ['ajv-keywords', 'htmlnano']   
dependent = {}
for pkg in pkgs:
    dependent[pkg] = []
    for obj in decode_stacked(fc):
        ind = 0
        try: versions = list(obj['versions'].keys())
        except: continue
        try: release_dates = obj['time']
        except: continue
        info = {key: {'release_date': release_dates[key]} for key in release_dates if key not in ['created', 'modified']}
        for key in versions:
            try:
                info[key]['dependencies'] = list(obj['versions'][key]['dependencies'].keys())
                if pkg in info[key]['dependencies']: ind = 1
            except: continue
        if ind :
            dependent[pkg].append(obj['name'])

In [36]:
import pymongo


dbname = "NPMvulnerabilities"
collname = "dependents"
client = pymongo.MongoClient("da1.eecs.utk.edu")

db = client[dbname]
coll = db[collname]

for k in dependent.keys():
    coll.insert_one({'name': k, 'dep': dependent[k]})

In [37]:
import pymongo


dbname = "NPMvulnerabilities"
collname = "dependents"
client = pymongo.MongoClient("da1.eecs.utk.edu")

db = client[dbname]
coll = db[collname]

for r in coll.find():
    print(r['name'])
    print(r['dep'])

ajv-keywords
['jsonscript-proxy', 'senasaikou-yeoman', 'webpack-egoist', 'alex-d.js', 'advanced-image-loader', 'webpack-addons-ylvis', 'primeng-custom', 'jicli', '@lgeiger/electron-builder', '@touched/map-tool-project', 'generator-arc', 'node-private-tools', 'fhir2', 'webpack-ie8', 'mocoolka-tools', 'misstep', '@cysonius/schema-utils', 'ng2-dnd-kokatsuna', 'qweb-client', '@johnf/electron-builder', 'qor', 'json-framework', 'hy-checkbox', 'babel-plugin-react-native-css', 'webpack-universal', 'babel-plugin-react-css-modules-avion', 'general-req', '@ngxvoice/ngx-voicelistner', 'electron-builder-admin', 'iqm', 'freemamba', 'isomorphic-webpack', 'webpack-tal', 'search-list-react', 'sea-floor', 'carousel-react', 'nicholas_util', 'search-input-react', 'json-injector', 'es-webpack', '@frctl/support', 'ajv-interface-keywords', 'stygian', 'lrbceshi', 'gh-lint', '@jose_santacruz/middy', '@ddder/webpack', 'vbot', 'outils-ren', 'canvas-fingerprint', 'firebase-forum', 'firebase-feedback', 'egeria-mer

In [146]:
import time, requests, sys, pymongo, json, datetime


dbname = "NPMvulnerabilities"
collname = "pkg_dl"
client = pymongo.MongoClient("da1.eecs.utk.edu")

db = client[dbname]
coll = db[collname]

#pkgs = ['ajv-keywords', 'htmlnano']
#pkgs = ['htmlnano']
pkgs = ['user-info', 'exec-file-sync', 'babylon-walk', 'speedtest-net', 'dns-then', 'node-then', '@babel/polyfill', '@babel/runtime', 'dvp-mongomodels', '@babel/helper-builder-react-jsx', 'doofinder', 'highland-errors-to', 'highland-process', '@cnbritain/merlin-www-section-card-list', '@cnbritain/merlin-www-tag-header', '@cnbritain/merlin-www-card-list', '@cnbritain/merlin-www-top-stories', 'uapi-json', '@beisen/ethos', '@cnbritain/merlin-www-card', '@babel/helper-replace-supers', 'bfj-node4', 'highland-errors-to', 'highland-process', '@particles/spinner', 'cli-ngy', '@cnbritain/merlin-www-card', 'neutrino-middleware-named-modules', 'chrome-pool', '@cnbritain/merlin-www-topic-strip', 'awayjs-display', '@babel/helper-replace-supers', 'cloudcms-server', 'authenticator-cli', 'authenticator', 'bfj-node4', 'chrome-runner', 'pino-toke', 'stencila', 'sqlite3-transactions', 'domotz-ip-address', 'authenticator-cli', 'safe-regex', 'easynode', 'authenticator', 'p-map-series', 'chunky-stream', 'expresser', 'semver-intersect', 'webpack-cli']
start_date = datetime.datetime(2015, 3, 5)
delta = datetime.timedelta(days=6)
end_date = start_date + delta
dt_range = []
final_date = datetime.datetime(2018, 10, 3)

while start_date < final_date:
    if end_date > final_date : end_date = final_date
    dt_range.append((start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d')))
    start_date = end_date + datetime.timedelta(days=1)
    end_date = start_date + delta


# get data
base_url = 'https://api.npmjs.org/downloads/point/'
l = len(pkgs)
print(l)
for pkg in pkgs:
    dl = []
    for r in dt_range:
        url = base_url+r[0]+':'+r[1]+'/'+pkg
        try: r = requests.get(url)
        except: print('Timeout '+ pkg+'\n')
        
        try:
            result = r.content
            try:
                rj = json.loads(result.decode('utf-8', errors='ignore'))
                rj.pop('package', None)
                if(rj['start'] != '2018-05-24' and rj['start'] != '2018-05-31' and rj['start'] != '2018-08-02' and rj['start'] != '2018-08-23'):
                    dl = dl + [rj]
            except: print('DecodeError '+ pkg+'\n')
        except: print('BadURL '+ pkg+'\n')
    #ent = {pkg: dl}    
    ent = {'name':pkg, 'dl': dl}

    coll.insert_one(ent)
    #print(ent)

50
DecodeError dns-then



In [ ]:
import math
total_list = []
start_date = []
end_date = []
download_list = []

for r in coll.find():
    download_list = []
    start_date = []
    ent = r['dl']
    for k in ent:
        download_list.append(k['downloads'])
        start_date.append(k["start"])
#map, reduce, filter, lambda
    log_list = [math.log(x+1) for x in download_list]
    difference_list = []
    for i in range(1,len(log_list)-1):
        dif = log_list[i] - log_list[i-1]
        difference_list.append( (log_list[i] - log_list[i-1]))
        if dif > 7:
            print("Start date of spike for " + r['name'] +": " +start_date[i])
        if dif < -7:
            print("Start date of fall for " + r['name'] +": " +start_date[i])
    #print(difference_list)
#print(log_list)
#print(download_list)


In [69]:
import time, requests, sys, pymongo, json, datetime


dbname = "NPMvulnerabilities"
collname = "dep_dl"
client = pymongo.MongoClient("da1.eecs.utk.edu")

db = client[dbname]
coll = db[collname]

#pkgs = ['ajv-keywords', 'htmlnano']
start_date = datetime.datetime(2015, 3, 1)
delta = datetime.timedelta(days=6)
end_date = start_date + delta
dt_range = []
final_date = datetime.datetime(2018, 10, 1)

while start_date < final_date:
    if end_date > final_date : end_date = final_date
    dt_range.append((start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d')))
    start_date = end_date + datetime.timedelta(days=1)
    end_date = start_date + delta


# get data
base_url = 'https://api.npmjs.org/downloads/point/'
l = len(pkgs)
print(l)
for k in dependent.keys():
    for i in dependent[k]:
        dl = []
        for r in dt_range:
            url = base_url+r[0]+':'+r[1]+'/'+i
            try: r = requests.get(url)
            except: print('Timeout '+ i+'\n')
        
            try:
                result = r.content
                try:
                    rj = json.loads(result.decode('utf-8', errors='ignore'))
                    rj.pop('package', None)
                    dl = dl + [rj]
                except: print('DecodeError '+ i+'\n')
            except: print('BadURL '+ i+'\n')
        #ent = {pkg: dl}    
        ent = {'name':i, 'dl': dl}
        coll.insert_one(ent)
        print(ent)

1


TypeError: Can't convert 'set' object to str implicitly

In [166]:
import math, time, requests, sys, pymongo, json, datetime

dbname = "NPMvulnerabilities"
collname = "pkg_dl"
client = pymongo.MongoClient("da1.eecs.utk.edu")

db = client[dbname]
coll = db[collname]

rise_list = {}
fall_list = {}
for r in coll.find():
    if not r['name'] in white_pkgs: 
        continue
        
    print("for package " + r['name'] + ":")
    download_list = []
    start_date = []
    rise_list[r['name']] = []
    fall_list[r['name']] = []
    ent = r['dl']
    for k in ent:
        download_list.append(k["downloads"])
        start_date.append(k["start"])
            
    for x in range(1,len(download_list) -1):
        if download_list[x] > 300:
            if (download_list[x] > 0) and (download_list[x-1] > 0):
                pdif = (download_list[x] - download_list[x-1]) / download_list[x-1]
                if (.3 <= pdif) and ((max(download_list) * .3)  < download_list[x]):
                    rise_list[r['name']].append(start_date[x])
                    print("We have a rise for " + r['name'] + " at " + start_date[x])
                if (-.3 >= pdif) and ((max(download_list) * .3) < download_list[x-1]):
                    fall_list[r['name']].append(start_date[x])
                    print("We have a fall for " + r['name'] + " at " + start_date[x])


for package ajv-keywords:
We have a rise for ajv-keywords at 2018-01-04
for package htmlnano:
We have a rise for htmlnano at 2017-12-21
We have a rise for htmlnano at 2017-12-28
We have a fall for htmlnano at 2018-01-04
for package object.map:
We have a rise for object.map at 2017-12-14
We have a rise for object.map at 2018-01-04
for package babylon-walk:
We have a rise for babylon-walk at 2017-12-21
We have a rise for babylon-walk at 2017-12-28
We have a fall for babylon-walk at 2018-01-04
for package wix-style-react:
We have a rise for wix-style-react at 2018-05-03
We have a fall for wix-style-react at 2018-05-10
for package threads:
We have a rise for threads at 2017-12-21
We have a fall for threads at 2018-01-04
for package cloudscraper:
We have a rise for cloudscraper at 2017-12-21
We have a fall for cloudscraper at 2018-01-04
for package user-info:
We have a rise for user-info at 2018-01-04
We have a rise for user-info at 2018-07-12
for package exec-file-sync:
We have a rise for 

In [167]:
for r in rise_list:
    #print("We have rises for " + r + " at " + str(rise_list[r]))
    for m in rise_list[r]:
        #print("We have a rise for " + r + " at " + m)
        print(r + ", " + m + ", " + "rise")
        
for r in fall_list:
    #print("We have falls for " + r + " at " + str(fall_list[r]))
    for m in fall_list[r]:
        print(r + ", " + m + ", " + "fall")

safe-regex, 2018-02-22, rise
highland-errors-to, 2018-01-11, rise
highland-errors-to, 2018-03-29, rise
highland-errors-to, 2018-07-12, rise
cloudscraper, 2017-12-21, rise
ajv-keywords, 2018-01-04, rise
wix-style-react, 2018-05-03, rise
babylon-walk, 2017-12-21, rise
babylon-walk, 2017-12-28, rise
awayjs-display, 2018-02-22, rise
node-then, 2017-12-21, rise
chrome-runner, 2018-02-01, rise
chrome-runner, 2018-02-22, rise
semver-intersect, 2018-02-15, rise
authenticator, 2018-02-08, rise
authenticator, 2018-02-15, rise
authenticator, 2018-03-22, rise
chrome-pool, 2018-02-22, rise
exec-file-sync, 2015-08-20, rise
p-map-series, 2018-09-20, rise
user-info, 2018-01-04, rise
user-info, 2018-07-12, rise
highland-process, 2018-01-11, rise
highland-process, 2018-03-29, rise
highland-process, 2018-07-12, rise
pino-toke, 2017-10-26, rise
pino-toke, 2017-11-09, rise
pino-toke, 2018-02-22, rise
pino-toke, 2018-03-08, rise
pino-toke, 2018-03-29, rise
pino-toke, 2018-04-19, rise
pino-toke, 2018-05-10, 

In [147]:
#revert to previous version and gets the old format of the versions. Gets dependent list with version history
#pkgs = ['ajv-keywords']  
dbname = "NPMvulnerabilities"
collname = "pkg_dl"
client = pymongo.MongoClient("da1.eecs.utk.edu")

db = client[dbname]
coll = db[collname]

dependent1 = {}
for r in coll.find():
    pkg = r['name']
    dependent1[pkg] = []   
    for obj in decode_stacked(fc):
        ind = 0
        try: versions = list(obj['versions'].keys())
        except: continue
        try: release_dates = obj['time']
        except: continue
        info = {key: {'release_date': release_dates[key]} for key in release_dates if key not in ['created', 'modified']}
        for key in versions:
            try:
                info[key]['dependencies'] = list(obj['versions'][key]['dependencies'].keys())
                if pkg in info[key]['dependencies']: ind = 1
            except: continue
        if ind :
            dependent1[pkg].append({obj['name']: info})

In [45]:
for k in dependent1.keys():
    print(k)
    print(dependent1[k][1:2])
    print("len = ", len(dependent1[k]))

ajv-keywords
[{'senasaikou-yeoman': {'1.0.0': {'dependencies': ['limiter', 'css', 'delayed-stream', 'is-path-in-cwd', 'promise.pipe', 'babel-plugin-transform-flow-strip-types', 'babel-plugin-transform-es2015-duplicate-keys', 'text-table', 'to-fast-properties', 'babylon', 'engine.io', 'postcss-svgo', 'resolve', 'babel-messages', 'babel-code-frame', 'gulp-autoprefixer', 'slash', 'through2-filter', 'options', 'postcss-discard-overridden', 'babel-helper-builder-react-jsx', 'bin-check', 'object.pick', 'levn', 'qs', 'strip-bom-stream', 'flatten', 'tfunk', 'form-data', 'ajv-keywords', 'yargs-parser', 'gulp-plumber', 'cryptiles', 'acorn-jsx', 'is-number', 'error-ex', 'immutable', 'filenamify', 'es6-iterator', 'strip-ansi', 'parse-glob', 'defaults', 'cross-spawn-async', 'semver', 'ansi-regex', 'json3', 'clone', 'dev-ip', 'extend', 'xmlhttprequest-ssl', 'underscore', 'vendors', 'fs-extra', 'lodash._reinterpolate', 'loose-envify', 'fs-exists-sync', 'sigmund', 'component-inherit', 'babel-core', 'l

In [144]:
# dict - {target package : [(dependent package, release)]}
from datetime import datetime as dt
delrl = {}
adrl = {}
all_del_deps =[]
for k in dependent1.keys():
    adrl[k] = [] #list of all versions with dependency where previous version did not have it
    delrl[k] = [] #list of all version without dependency where previous version did have it
    for l in dependent1[k]:
        d = list(l.keys())[0]
        #print(k + ' : ' + d)
        v = l[d]
        rdt_l = {}


        for rk in v.keys():
            r = v[rk]

            rdt = r['release_date'][:10]+' '+r['release_date'][11:19]
            print(rdt)
            rdt = dt.strptime(rdt , "%Y-%m-%d %H:%M:%S")
            try:
                if k in r['dependencies']: rdt_l[rdt] = ['Y',rk]
                else: rdt_l[rdt] = ['N', rk]
            except:
                continue

        rdt_s = sorted(rdt_l.keys())
        fadd = 0

        for r in rdt_s:
            if fadd == 0 and rdt_l[r][0] == 'Y':
                if r.date() < dt(2015,3,1).date(): t = dt(2015,3,1).date()
                else: t = r.date()
                adrl[k].append([d,  rdt_l[r][1], str(r)])
                print(k + ' : ' + d + ' ' + rdt_l[r][1] + ' ' + str(r))
                fadd = 1
            if fadd and rdt_l[r][0] == 'N':
                if r.date() < dt(2015,3,1).date(): t = dt(2015,3,1).date()
                else: t = r.date()
                delrl[k].append([d, rdt_l[r][1], str(r)])
                #print(k + ' : ' + d + ' ' + rdt_l[r][1] + ' ' + str(r))
                all_del_deps.append(d)
                fadd = 0

2016-06-19 20:02:13
2016-06-11 22:09:59
2016-06-12 20:28:36
2016-06-11 21:10:29
2016-07-03 18:11:48
ajv-keywords : jsonscript-proxy 0.1.0 2016-06-11 21:10:29
2016-10-23 12:28:02
2016-10-23 14:21:58
2016-10-24 02:12:17
2016-10-29 09:20:06
2016-10-23 14:14:05
2016-10-30 12:10:22
ajv-keywords : senasaikou-yeoman 1.0.0 2016-10-23 12:28:02
2016-12-23 15:05:08
ajv-keywords : webpack-egoist 2.2.0-rc.2 2016-12-23 15:05:08
2017-03-23 00:37:23
2017-03-23 01:32:44
2017-03-23 01:46:00
2017-03-23 01:31:59
2017-03-23 01:29:57
ajv-keywords : alex-d.js 1.0.0 2017-03-23 00:37:23
2017-04-02 19:13:04
ajv-keywords : advanced-image-loader 1.0.0 2017-04-02 19:13:04
2017-01-28 16:51:59
2017-01-28 15:16:05
2017-01-28 16:56:25
2017-01-28 16:44:35
2017-01-28 16:37:23
2017-01-28 15:21:50
2017-02-03 22:11:40
2017-01-05 07:07:21
2017-01-28 16:28:33
2017-01-28 15:51:48
2017-03-21 21:01:02
2017-03-21 20:55:14
2017-01-28 16:45:24
2017-01-28 16:47:23
2017-04-05 10:49:14
2017-01-28 15:24:36
2017-01-28 15:08:06
2017-03-

2017-07-11 12:46:47
2017-06-12 09:40:35
2017-06-21 11:33:48
2017-06-09 06:29:51
2017-10-12 17:12:58
2018-02-19 12:35:50
2017-06-07 15:29:16
2017-06-06 15:22:02
2017-10-17 14:17:24
2017-06-27 08:37:23
2018-02-19 11:35:09
2017-06-09 09:51:00
2017-08-18 10:27:42
2017-06-10 14:39:22
2017-10-12 06:14:31
2017-10-17 10:06:03
2017-10-03 11:51:12
2017-06-10 13:06:47
2017-06-12 09:12:44
2017-11-14 11:03:16
2017-06-12 11:17:28
2017-07-16 11:45:35
2017-06-06 09:20:25
2017-10-17 08:18:01
2017-06-21 14:33:05
2017-10-13 09:47:18
2017-06-06 14:04:02
2017-06-11 10:14:42
2017-06-08 16:13:10
2017-06-12 10:58:47
2017-06-09 10:26:54
2017-06-12 11:02:53
2017-11-11 12:42:43
2017-06-14 10:01:28
2017-10-19 12:57:25
2017-07-11 13:12:05
2017-06-05 14:11:41
2017-07-11 17:54:03
2017-10-17 11:33:13
2017-08-18 10:25:56
2017-06-10 15:06:41
2017-06-08 13:43:59
2017-10-13 05:44:31
2018-08-21 11:16:04
2017-09-20 20:51:43
2017-10-15 08:27:11
2018-01-29 15:26:23
2017-10-19 12:14:03
2017-10-13 09:21:57
2018-04-12 09:55:11


2016-12-25 19:56:45
2015-11-18 22:44:00
2016-06-03 19:31:37
2016-04-29 18:39:36
2016-07-02 18:29:20
2017-02-15 20:53:33
2017-06-13 05:57:19
2016-09-19 14:00:27
2017-08-21 07:44:59
2016-11-07 13:52:42
2017-11-19 11:24:59
2018-05-17 16:34:05
2016-04-09 14:53:20
2017-04-12 08:05:44
2016-11-26 09:23:02
2015-05-28 21:36:55
2016-09-10 09:15:25
2018-06-30 09:39:50
2018-01-06 10:09:40
2018-01-06 10:59:48
2018-08-14 08:30:13
2017-09-10 07:33:58
2018-02-10 10:53:48
2017-06-16 16:12:55
2017-03-17 07:21:21
2017-02-24 13:14:29
2017-02-01 08:05:00
2016-06-19 14:30:22
2016-11-09 08:43:35
2017-11-11 13:04:31
2017-06-01 17:20:17
2017-05-21 07:39:57
2018-07-20 08:10:44
2017-08-28 12:38:20
2017-11-22 16:58:57
2016-08-02 06:01:01
2016-06-23 17:04:36
2016-04-14 12:28:33
2017-10-18 08:17:57
2018-09-05 19:01:52
2017-06-22 17:16:57
2017-09-26 07:25:35
2017-04-26 07:35:40
2017-04-23 07:01:19
2017-01-16 20:42:08
2018-07-17 06:37:51
2016-11-04 04:23:11
2018-04-19 07:11:41
2016-03-23 09:36:47
2017-11-04 09:37:38


2018-09-01 17:17:29
2018-08-04 14:28:53
2017-12-21 13:01:29
2017-12-24 01:04:28
2018-01-24 15:46:50
2018-06-20 08:31:53
2018-02-02 09:11:28
2018-01-28 17:19:20
ajv-keywords : @syncano/validate 0.10.0 2018-03-08 18:48:37
2018-09-07 08:34:13
2018-09-07 07:24:58
2018-09-07 09:13:48
2018-09-07 07:44:51
2018-09-07 09:27:13
2018-09-07 09:29:06
2018-09-07 10:03:57
2018-09-07 09:54:45
2018-09-07 09:38:55
2018-09-07 09:34:18
ajv-keywords : ylztest 0.0.1 2018-09-07 08:34:13
2018-08-29 15:42:54
2018-03-05 09:52:26
2018-06-18 09:45:23
2018-09-05 13:02:51
2018-04-18 07:16:39
2018-06-18 08:41:35
2018-07-20 09:54:36
2018-08-30 11:55:57
2018-06-14 06:22:28
2018-01-19 10:25:27
2018-06-13 08:42:33
2018-09-06 09:12:47
2018-04-17 10:30:49
2018-02-08 16:54:12
2018-05-28 11:11:59
2018-01-24 09:28:30
2018-02-25 20:15:40
2018-04-18 09:51:22
2018-07-25 12:12:15
2018-04-04 13:21:51
2018-04-17 10:02:13
2018-06-13 14:48:29
2018-08-20 18:14:15
2018-07-23 16:24:55
2018-07-23 15:46:58
2018-03-29 20:40:58
2018-08-28 

In [182]:
for k in adrl:
    print('package add dates for ' + k)
    for r in range(0, len(adrl[k])):
        print(adrl[k][r])
for k in delrl:
    print('package drop dates for ' + k)
    for r in range(0, len(delrl[k])):
        print(delrl[k][r])

package add dates for ajv-keywords
['jsonscript-proxy', '0.1.0', '2016-06-11 21:10:29']
['senasaikou-yeoman', '1.0.0', '2016-10-23 12:28:02']
['webpack-egoist', '2.2.0-rc.2', '2016-12-23 15:05:08']
['alex-d.js', '1.0.0', '2017-03-23 00:37:23']
['advanced-image-loader', '1.0.0', '2017-04-02 19:13:04']
['webpack-addons-ylvis', '0.0.27', '2017-02-05 14:01:06']
['primeng-custom', '4.0.0-beta.1', '2017-04-06 09:34:45']
['jicli', '0.0.1', '2017-03-30 01:47:09']
['@lgeiger/electron-builder', '1.0.0', '2017-04-09 21:24:46']
['@touched/map-tool-project', '0.1.0', '2017-04-23 15:24:49']
['generator-arc', '0.0.1', '2016-10-14 13:11:12']
['node-private-tools', '2.0.0', '2017-05-09 04:38:12']
['fhir2', '1.0.0', '2017-05-23 20:29:06']
['webpack-ie8', '2.1.0-beta.26', '2016-11-14 05:38:31']
['mocoolka-tools', '0.5.22', '2017-04-25 19:39:45']
['misstep', '1.0.0', '2017-06-14 19:34:09']
['@cysonius/schema-utils', '0.0.14', '2017-05-06 16:41:19']
['ng2-dnd-kokatsuna', '1.0.0', '2017-06-17 16:54:40']
['q

In [183]:
for r in rise_list:
    #print("We have rises for " + r + " at " + str(rise_list[r]))
    for m in rise_list[r]:
        temp1 = dt.strptime((str(m) + " 00:00:00"), "%Y-%m-%d %H:%M:%S")
        temp1 = rdt - temp1
        print("Time difference " + str(temp1))
        
for r in fall_list:
    #print("We have falls for " + r + " at " + str(fall_list[r]))
    for m in fall_list[r]:
        print("We have a fall for " + r + " at " + m)

NameError: name 'rise_list' is not defined

In [145]:
for i in rise_list:
    for j in rise_list[i]:
        for k in adrl:
            for l in range(0, len(adrl[k])):
                dif = dt.strptime((str(adrl[k][l][2])), "%Y-%m-%d %H:%M:%S")
                dif = j - dif
                if (dif < datetime.timedelta(days=14)):
                    print("Time difference " + str(dif))

TypeError: unsupported operand type(s) for -: 'str' and 'datetime.datetime'